# FX vol trading strategy - implementation

In [ ]:
# Import libraries
import pandas as pd
import numpy as np

### Algorithm description (WIP..)

1) For a given trading pair, say USDRUB, fit the appropriate GARCH model for time t = 0 until time t = T
2) Standing at time T, forecast the 30-day volatility. If the forecast is above market pricing, enter long straddle, if below then enter a short straddle. If not signficantly different, do nothing. 